# konlpy 형태소 분석기 성능비교

In [1]:
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Hannanum
#from konlpy.tag import Mecab
import MeCab
from konlpy.tag import Okt
import re

# 형태소 분석을 위한 객체 생성.
kkma = Kkma()
komoran = Komoran()
hannanum = Hannanum()
mecab = MeCab.Tagger()
#mecab = Mecab()
okt = Okt()

C:\Users\MyCom\anaconda3\envs\text_analysis\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
"""
    Parsing 규칙의 문제점, split을 "," 기준으로 하는데, token이 "," 인 경우에는 쉼표만 잘려서 나오기 때문에, 
    + "%," 같이 특수문자와 쉼표가 같이 등장하는 경우도 생각해주어야 함.
    
    (",", "SC") 의 원래 튜플이 만들어지지 않음.
    
    명사 분석의 경우 해당 토큰이 필요하지 않으니 pass
    
    형태소 분석과 POS tagging의 경우 해당 토큰이 필요하므로, token[0]이 ' 인 경우엔 따로 (",", "SC")를 집어 넣어줘야함.
"""


def mecab_nouns(text):
    nouns = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    # 추출한 token중에 POS가 명사 분류에 속하는 토큰만 선택.
    for token in temp:
        if token[1] == "NNG" or token[1] == "NNP" or token[1] == "NNB" or token[1] == "NNBC" or token[1] == "NP" or token[1] == "NR":
             nouns.append(token[0])
        
    return nouns

def mecab_morphs(text):
    morphs = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        
    # 추출한 token중에 문자열만 선택.
    for token in temp:
        morphs.append(token[0])
    
    return morphs

def mecab_pos(text):
    pos = []
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+") 
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    pos = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
        #group(0)으로 string으로 만들어 준다
    return pos

* 정규 표현식은 특정한 규칙을 가진 문자열의 패턴을 표현하는 데 사용하는 표현식(Expression)으로 텍스트에서 특정 문자열을 검색하거나 치환할 때 흔히 사용된다. 

 

## 문장 분석 품질 비교

> ### 1. 띄어쓰기가 제대로 되어있지 않은 문장

In [3]:
text = "아버지가방에들어가신다"

In [4]:
# 꼬꼬마 형태소 분석 결과
kkma.pos(text)

[('아버지', 'NNG'),
 ('가방', 'NNG'),
 ('에', 'JKM'),
 ('들어가', 'VV'),
 ('시', 'EPH'),
 ('ㄴ다', 'EFN')]

In [5]:
# 코모란 형태소 분석 결과
komoran.pos(text)

[('아버지', 'NNG'),
 ('가방', 'NNP'),
 ('에', 'JKB'),
 ('들어가', 'VV'),
 ('시', 'EP'),
 ('ㄴ다', 'EC')]

In [6]:
hannanum.pos(text)

[('아버지가방에들어가', 'N'), ('이', 'J'), ('시ㄴ다', 'E')]

In [7]:
mecab_pos(text)

[('아버지', 'NNG'),
 ('가', 'JKS'),
 ('방', 'NNG'),
 ('에', 'JKB'),
 ('들어가', 'VV'),
 ('신다', 'EP')]

In [8]:
okt.pos(text)

[('아버지', 'Noun'), ('가방', 'Noun'), ('에', 'Josa'), ('들어가신다', 'Verb')]

> ### 2. 오탈자 때문에 불완전한 문장

In [9]:
text2 = "ㄱㅐㄴㅏ리가 피어있는 동산에 누워있고싶ㄷㅏ"

In [10]:
# 꼬꼬마 형태소 결과 출력하기.

print("꼬꼬마 : %s\n" % kkma.pos(text2))
print("코모란 : %s\n" % komoran.pos(text2))
print("한나눔 : %s\n" % hannanum.pos(text2))
print("mecab : %s\n" % mecab_pos(text2))
print("Okt : %s\n" % okt.pos(text2))

꼬꼬마 : [('ㄱㅐㄴㅏ리', 'UN'), ('가', 'JKS'), ('피', 'VV'), ('어', 'ECD'), ('있', 'VXV'), ('는', 'ETD'), ('동산', 'NNG'), ('에', 'JKM'), ('눕', 'VV'), ('어', 'ECS'), ('있', 'VV'), ('고', 'ECE'), ('싶ㄷㅏ', 'UN')]

코모란 : [('개나리', 'NNP'), ('가', 'JKS'), ('피', 'VV'), ('어', 'EC'), ('있', 'VX'), ('는', 'ETM'), ('동산', 'NNP'), ('에', 'JKB'), ('눕', 'VV'), ('어', 'EC'), ('있', 'VX'), ('고', 'EC'), ('싶', 'VX'), ('다', 'EC')]

한나눔 : [('ㄱㅐㄴㅏ리', 'N'), ('가', 'J'), ('피', 'P'), ('어', 'E'), ('있', 'P'), ('는', 'E'), ('동산', 'N'), ('에', 'J'), ('누워있고싶ㄷㅏ', 'N')]

mecab : [('ㄱ', 'NNG'), ('ㅐㄴㅏ리가', 'UNKNOWN'), ('피', 'VV'), ('어', 'EC'), ('있', 'VX'), ('는', 'ETM'), ('동산', 'NNG'), ('에', 'JKB'), ('누워', 'VV'), ('있', 'VX'), ('고', 'EC'), ('싶', 'VX'), ('ㄷ', 'NNG'), ('ㅏ', 'UNKNOWN')]

Okt : [('ㄱㅐㄴㅏ', 'KoreanParticle'), ('리가', 'Noun'), ('피어있는', 'Verb'), ('동산', 'Noun'), ('에', 'Josa'), ('누워있고싶', 'Verb'), ('ㄷㅏ', 'KoreanParticle')]



> ### 3. 속도 비교

In [11]:
text3 = """5G 이동통신망을 빌려 사용하는 ‘5G 알뜰폰’이 올해 도입되고, 내년부터는 의무화된다.
정부는 알뜰폰 사업자(MNVO)가 통신사(MNO)에 통신망을 빌리는 비용(도매대가)을 지난해보다 큰 폭으로 낮춰, 알뜰폰 요금 인하를 유도하기로 했다. 하지만 줄어드는 알뜰폰 시장을 살릴 수 있을지는 지켜봐야 하는 상황이다.
과학기술정보통신부는 알뜰폰 활성화 추진대책을 25일 발표했다. 알뜰폰 가입자는 800만명으로 이동통신 시장의 12%를 차지한다. 2011년 출시 뒤 저렴한 요금제로 통신비 부담을 낮춰왔다. 하지만 지난해 5월 통신 3사가 준보편제 요금을 내놓은 이후 알뜰폰 이탈 현상이 지속되고 있다.
우선 올해 안에 3개 이상의 5G 알뜰폰이 시장에 나온다. 통신사가 5G망을 알뜰폰 사업자에게 도매 제공할지 여부는 통신사 자율로 정한다. 앞서 LG유플러스는 오는 10월 알뜰폰 사업을 시작하는 KB국민은행에 5G망을 제공한다고 밝힌 바 있다. SK텔레콤와 KT도 특정 제휴사를 선택해 올해 안에 5G 알뜰폰을 내놓기로 했다.
내년부터는 5G 알뜰폰 제공이 의무화된다. 지난 22일자로 종료된 도매제공 의무제도의 유효기간을 2022년 9월22일까지 연장하는 전기통신사업법 개정안이 국회에서 통과되면, 관련 고시를 개정해 SK텔레콤의 5G망 도매제공을 의무화하겠다는 것이다.
과기정통부 관계자는 “SK텔레콤이 자사와 계약을 맺은 13개 알뜰폰 사업자에게 5G망을 의무 제공하면, 그 외 31개의 알뜰폰 사업자들이 경쟁에서 밀릴 것을 우려해 KT와 LG유플러스도 5G망을 제공하게 될 것”이라고 내다봤다.
알뜰폰 사업자가 상품을 만드는 방식 크게 2가지다. 하나는 통신사로부터 음성·문자·데이터를 도매로 사들인 뒤 이를 바탕으로 통신사보다 저렴한 요금제를 내놓는 방식(종량제 도매제공)이다. 이를 위해 정부는 도매대가 인하율을 음성 17.8%, 데이터 19.2%, 단문메시지 1.15%로, 지난해 음성 15.1%, 데이터 19.1%, 단문메시지 1.13%에 비해 높여 잡았다.
또 다른 방식은 일정비용을 통신사에 내고 통신사의 정액 요금제를 그대로 판매하면서, 그 차액의 범위에서 저렴한 요금제를 내놓는 방식(수익배분 도매제공)이다. 정부는 SK텔레콤의 준보편 요금제인 ‘T플랜 요금제’를 알뜰폰 사업자가 재판매할 수 있게 했다. 기존에 SK텔레콤이 도매제공했던 ‘밴드데이터 요금제’의 최고구간의 대가도 1.5%포인트 낮췄다.
알뜰폰 업계는 대체로 반기는 분위기지만, 알뜰폰 시장을 살릴 수 있을지에는 의구심을 갖고 있다. 업계 관계자는 “도매대가 인하율이 크고, 5G망을 제공하는 것은 긍정적”이라면서도 “수익배분 도매제공의 의무화, 설비를 가진 업체에 대한 접속료 정산 도입 등의 제도적 개선이 필요하다”고 말했다."""

In [12]:
%%time
print(kkma.pos(text3)[:5])

[('5', 'NR'), ('G', 'OL'), ('이동', 'NNG'), ('통신망', 'NNG'), ('을', 'JKO')]
Wall time: 4.13 s


In [13]:
%%time
print(komoran.pos(text3)[:5])

[('5', 'SN'), ('G', 'SL'), ('이동', 'NNG'), ('통신망', 'NNG'), ('을', 'JKO')]
Wall time: 70 ms


In [14]:
%%time
print(hannanum.pos(text3)[:5])

[('5G', 'N'), ('이동통신망', 'N'), ('을', 'J'), ('빌리', 'P'), ('어', 'E')]
Wall time: 1.82 s


In [15]:
%%time
print(mecab_pos(text3)[:5])#5개만 짤라서 출력결과 확인 가능

[('5', 'SN'), ('G', 'SL'), ('이동', 'NNG'), ('통신망', 'NNG'), ('을', 'JKO')]
Wall time: 256 ms


In [21]:
%%time
print(mecab_nouns(text3))

['이동', '통신망', '사용', '알뜰', '폰', '올해', '도입', '내년', '의무', '정부', '알뜰', '폰', '사업자', '통신사', '통신망', '비용', '도매', '대가', '지난해', '폭', '알뜰', '폰', '요금', '인하', '유도', '알뜰', '폰', '시장', '수', '상황', '과학', '기술', '정보', '통신부', '알뜰', '폰', '활성', '추진', '대책', '일', '발표', '알뜰', '폰', '가입자', '만', '명', '이동', '통신', '시장', '차지', '년', '출시', '뒤', '요금제', '통신비', '부담', '지난해', '월', '통신', '사', '준보', '편제', '요금', '이후', '알뜰', '폰', '이탈', '현상', '지속', '올해', '안', '개', '이상', '알뜰', '폰', '시장', '통신사', '망', '알뜰', '폰', '사업자', '도매', '제공', '여부', '통신사', '자율', '유', '플러스', '월', '알뜰', '폰', '사업', '시작', '국민은행', '망', '제공', '바', '텔레콤', '특정', '휴사', '선택', '올해', '안', '알뜰', '폰', '내년', '알뜰', '폰', '제공', '의무', '일자', '종료', '도매', '제공', '의무', '제도', '유효', '기간', '년', '월', '일', '연장', '전기', '통신', '사업', '법', '개정안', '국회', '통과', '관련', '고시', '개정', '텔레콤', '망', '도매', '제공', '의무', '것', '과기', '정통부', '관계자', '텔레콤', '자사', '계약', '개', '알뜰', '폰', '사업자', '망', '의무', '제공', '외', '개', '알뜰', '폰', '사업자', '경쟁', '것', '우려', '유', '플러스', '망', '제공', '것', '알뜰', '폰', '사업자', '상품', '방식', '가지',

In [19]:
%%time
print(mecab_pos(text3))

[('5', 'SN'), ('G', 'SL'), ('이동', 'NNG'), ('통신망', 'NNG'), ('을', 'JKO'), ('빌려', 'VV'), ('사용', 'NNG'), ('하', 'XSV'), ('는', 'ETM'), ('‘', 'SY'), ('5', 'SN'), ('G', 'SL'), ('알뜰', 'NNG'), ('폰', 'NNG'), ('’', 'SY'), ('이', 'MM'), ('올해', 'NNG'), ('도입', 'NNG'), ('되', 'XSV'), ('고', 'EC'), (',', 'SC'), ('내년', 'NNG'), ('부터', 'JX'), ('는', 'JX'), ('의무', 'NNG'), ('화', 'XSN'), ('된다', 'XSV'), ('.', 'SF'), ('정부', 'NNG'), ('는', 'JX'), ('알뜰', 'NNG'), ('폰', 'NNG'), ('사업자', 'NNG'), ('(', 'SSO'), ('MNVO', 'SL'), (')', 'SSC'), ('가', 'JKS'), ('통신사', 'NNG'), ('(', 'SSO'), ('MNO', 'SL'), (')', 'SSC'), ('에', 'JKB'), ('통신망', 'NNG'), ('을', 'JKO'), ('빌리', 'VV'), ('는', 'ETM'), ('비용', 'NNG'), ('(', 'SSO'), ('도매', 'NNG'), ('대가', 'NNG'), (')', 'SSC'), ('을', 'JKO'), ('지난해', 'NNG'), ('보다', 'JKB'), ('큰', 'VA'), ('폭', 'NNG'), ('으로', 'JKB'), ('낮춰', 'VV'), (',', 'SC'), ('알뜰', 'NNG'), ('폰', 'NNG'), ('요금', 'NNG'), ('인하', 'NNG'), ('를', 'JKO'), ('유도', 'NNG'), ('하', 'XSV'), ('기', 'ETN'), ('로', 'JKB'), ('했', 'VV'), ('다', 'EF'), ('.

In [18]:
%%time
print(mecab_morphs(text3))

['5', 'G', '이동', '통신망', '을', '빌려', '사용', '하', '는', '‘', '5', 'G', '알뜰', '폰', '’', '이', '올해', '도입', '되', '고', ',', '내년', '부터', '는', '의무', '화', '된다', '.', '정부', '는', '알뜰', '폰', '사업자', '(', 'MNVO', ')', '가', '통신사', '(', 'MNO', ')', '에', '통신망', '을', '빌리', '는', '비용', '(', '도매', '대가', ')', '을', '지난해', '보다', '큰', '폭', '으로', '낮춰', ',', '알뜰', '폰', '요금', '인하', '를', '유도', '하', '기', '로', '했', '다', '.', '하', '지만', '줄어드', '는', '알뜰', '폰', '시장', '을', '살릴', '수', '있', '을지', '는', '지켜봐야', '하', '는', '상황', '이', '다', '.', '과학', '기술', '정보', '통신부', '는', '알뜰', '폰', '활성', '화', '추진', '대책', '을', '25', '일', '발표', '했', '다', '.', '알뜰', '폰', '가입자', '는', '800', '만', '명', '으로', '이동', '통신', '시장', '의', '12', '%', '를', '차지', '한다', '.', '2011', '년', '출시', '뒤', '저렴', '한', '요금제', '로', '통신비', '부담', '을', '낮춰', '왔', '다', '.', '하지만', '지난해', '5', '월', '통신', '3', '사', '가', '준보', '편제', '요금', '을', '내놓', '은', '이후', '알뜰', '폰', '이탈', '현상', '이', '지속', '되', '고', '있', '다', '.', '우선', '올해', '안', '에', '3', '개', '이상', '의', '5', 'G', '알뜰', '폰'

In [16]:
%%time
print(okt.pos(text3)[:5])

[('5', 'Number'), ('G', 'Alpha'), ('이', 'Determiner'), ('동', 'Modifier'), ('통신망', 'Noun')]
Wall time: 302 ms
